## E-tivity   - Binary Logistic regression - Task A
There are many situations where some $y$ should be a noisy
representation of some function of $x_1,\ldots,x_p$, but the 
response outcomes $y$ are binary.

Binary logistic regression corresponds to this probabilistic model

$$
p\left(y|w^T x \right)=\phi(w^T x)^{y}(1-\phi(w^T x ))^{1-y}  \text{ for } y=0,1
$$

where $w \in \mathbb{R}^p$ is the vector if coefficient and $\phi(\cdot)$ is some sigmoid function,e.g.,

* Logistic function: $\phi(x)=1/(1+e^{-x})$
* probit function: $\phi(x)=0.5(1+\text{erf}(x/\sqrt{2}))$


Given $N$ training data, the likelihood function assuming that all the observations in the sample are independently Bernoulli distributed is

$$
{\displaystyle {\begin{aligned}L(\theta \mid x)&=\Pr(Y\mid X;\theta )\\&=\prod_{i=1}^Np(y_{i}\mid x_{i};\theta )\\&=\prod _{i=1}^N\phi(w^T x_{i})^{y_{i}}(1-\phi(w^T x_{i}))^{(1-y_{i})}\end{aligned}}}
$$

Our goal is to find the maximum likelihood estimate of the vector of the parameters $w$, that is to solve the optimisation problem

$$
\hat{w} =\arg\max_{w \in \mathbb{R}^p} \prod _{i=1}^N\phi(w^T x_{i})^{y_{i}}(1-\phi(w^T x_{i}))^{(1-y_{i})}
$$

The product of probability (that are numbers between $0$ and $1$) can become very small for large $N$
and can lead to arithmetic underflow problems.
To avoid this problem, one typically maximises the logarithm of the likelihood function (log transform). In fact, one can prove that for any nonnegative function

$$
\arg\max_{x} f(x) = \arg\max_{x} \log(x)  
$$

and, therefore, we actually solve:

$$
\hat{w} =\arg\max_{w \in \mathbb{R}^p}  \sum _{i=1}^{N}y_{i}\log(\phi(w^T x_{i}))+(1-y_{i})\log(1-\phi(w^T x_{i}))
 $$

which is maximized using optimization techniques.

**Reguralisation**
In general-recipe ML, reguralisation is used to reduce the problem of overfitting. 
Therefore, actually the problem we aim to solve is

$$
\hat{w} =\arg\max_{w \in \mathbb{R}^p}  \sum _{i=1}^{N}y_{i}\log(\phi(w^T x_{i}))+(1-y_{i})\log(1-\phi(w^T x_{i})) + \alpha||w||_2^2
 $$
 
 where $ \alpha$ is the reguralisation constant (a design parameter that is usually selected via cross-validation, note that $ \alpha\geq0$) and $||w||_2^2=\sum_{=1}^{p} w_i^2$. 


We assume that you know how logistic regression works: (i) how to compute the predicted probability; (ii) how to return the predicted class.


## Task A
Part 1 Theorethical questions: 

    What type of optimisation problem is it?
    1. Is it univariate or multivariate?
    2. Is it linear or nonlinear?
    3. Is it convex or not? (google it and cite your font, how is the fast way to prove it?)
    4. Is it constrained?
    
Part 2: The goal of this part is to reimplement **totally** from scratch the class LogisticRegression, that mimics the corresponding class in `Sklearn`. 
* The class must be flexible: the user can choose any type of differentiable (w.r.t. the parameters $w$) sigmoid function and the class must work for any number of features $p$ (suggestion: use `autograd` library). 
* As optimizer you should use `local_descent` with  `line_research`.



**Submission (README):** Upload your notebook to the Gitlab repository and submit a post to your forum.
* Please follow this format for your filename: **CS5014_E-tivity1A -YourName_YourStudentID.ipynb** and add your full name and student number at the top of your submitted notebook.
* Your forum post should include a few words about your submission and a link to your notebook in Gitlab.
* The notebook must only include the source code of the implemented class and optimization algorithm
* A short example about how to use it (e.g., how to run your code on the iris dataset, see the example below) so your peers can understand how to run your code. 
* You must **not** include any tests you performed to verify that your code is properly working.

In [1]:
#Skeleton code
class Generic_LogisticRegression():
    def __init__(self, θ, sigmoid, α=1.0):
        self.θ = θ #initial value of the parameter
        self.sigmoid = sigmoid #a sigmoid function that squezes the input into [0,1]
        self.α = α #reguralisation parameter
        
    def fit(self,X,y,optimization_method,Optimparam): 
        
        #first define the cost function as given above
        def cost(θ):
            z = np.dot(X, θ)
            sig=self.sigmoid(z)
            p1 = y * np.log(sig)
            p0 = (1 - y) * np.log(1 - sig)
            return (-np.sum(p1 + p0))  + self.α * sum(np.square(θ[:]))
        
        
        #automatic diff gradient function

        gradcost=grad(cost)

        #call the optimizer
        θ = optimization_method(self.θ, cost, gradcost, steps=Optimparam['steps'], α=self.α, ϵ_x =  Optimparam['ϵ_x'], ϵ_d =  Optimparam['ϵ_d'], plotting=False)
        self.θ = θ[-1]
        ;
    
    def predict_proba(self, X):
        #predict probability of the class

        return self.sigmoid(np.dot(X, self.θ))
        ;
        
    def predict(self, X, threshold=0.5):
        #predict class

        return [1 if x >= threshold else 0 for x in self.predict_proba(X)]
        ;

#your implementation of the optimization algorithm

def minimize(β,cost, gradient,steps, α, ϵ_x=0.0001, ϵ_d=0.0001, plotting=False):
    
    # the optimisation algorithm we will use is local descent with line search 
    def line_search(f, x, d, ϵ_x, ϵ_d):
        term = False
        if np.linalg.norm(d) < ϵ_d:
            print("Gradient norm below threshold")
            term = True
            return x, term 
        objective = lambda α : f(x + α*d)
        a, b = bracket_minimum(objective) #function defined below
        a, b  = golden_section_search(objective, a, b, max_iter = 5, plotting=False) #function defined below
        α = (a+b)/2
        if np.linalg.norm(x+a*d-x)<ϵ_x:
            print("Step Tolerance below threshold")
            term = True
        return x + α*d, term

    def local_descent(β,cost, gradient,steps, α, ϵ_x, ϵ_d,  plotting=False):
        Tmp = [β]
        for iteration in range(steps):
            d = gradient(β)
            d = -d/np.linalg.norm(d) 
            #print(β,d)
            β, term = line_search(cost, β, d, ϵ_x=ϵ_x, ϵ_d=ϵ_d)
            if term == True:
                break
            Tmp.append(β)
        #print(np.array(Tmp))
        return Tmp


    #bracket_minimum required for line search function
    def bracket_minimum(f, β=0, s=1e-2, k=2.0): #From the Book, pag.36
        a, ya = β, f(β)
        b, yb = a + s, f(a + s)
        if yb > ya:
            a, b = b, a
            ya, yb = yb, ya
            s = -s
        while True:
            c, yc = b + s, f(b + s)
            if yc > yb:
                return (a, c) if a<c else (c, a)        
            a, ya, b, yb = b, yb, c, yc
            s *= k

    from math import sqrt
    ϕ = (1 + sqrt(5))/2

    #golden section search also required for line search
    def golden_section_search(f, a, b, max_iter, plotting=True): #from the Book pag.41
        a0 =a
        b0=b
        ρ =  ϕ-1
        d = ρ * b + (1 - ρ)*a
        yd = f(d)
        for i in range(max_iter-1):
            c = ρ*a + (1 - ρ)*b
            yc = f(c)
            if yc < yd:
                b, d, yd = d, c, yc
            else:
                a, b = b, c   
            if plotting==True:
                plt.figure()
                xx = np.linspace(a0,b0,100)
                plt.plot(xx,f(xx))
                plt.scatter(np.array([a,b]),np.array([a,b])*0)
                plt.scatter(np.array([a,b]),np.array([f(a),f(b)]))

        return (a, b) if a<b else (b, a)

    return local_descent(β,cost, gradient,steps,α , ϵ_x, ϵ_d)

#### Hints

In [2]:
from autograd.scipy.special import erf
import autograd.numpy as np  # 
from autograd import grad    #

graderf = grad(erf)
graderf(np.array([1.0]))

array([0.4151075])

In [3]:
   
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
ind = np.where((y==1) | (y==2))[0] # we only have two classes now
X = X[ind,:]
y = y[ind]
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])

clf.predict_proba(X[:4, :])

clf.predict(X)


from sklearn.metrics import confusion_matrix
confusion_matrix(y, clf.predict(X))

array([[47,  3],
       [ 1, 49]], dtype=int64)

In [ ]:
# Repeat using our Generic_LogisticRegression() rather than sklearns

#Define the sigmoid function we'll use
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))


# Get initial values for the weights 
θ = np.random.rand(1, X.shape[1]).reshape(-1)

# define parameters for Optimparam
Optimparam = {
    'steps':5000, 
    'ϵ_x':0.00001, 
    'ϵ_d':0.00001}

# Fit the model 
GLR = Generic_LogisticRegression(θ, sigmoid, α=0.001)
GLR.fit(X, y, minimize, Optimparam)

# predict probabilities
ypred_proba = GLR.predict_proba(X)

# predict classes
ypred = GLR.predict(X)
np.array([ypred])

C:\ProgramData\Anaconda3\envs\tensorflow-22\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
<ipython-input-1-d8979604d63c>:15: RuntimeWarning: invalid value encountered in multiply
  p0 = (1 - y) * np.log(1 - sig)
<ipython-input-1-d8979604d63c>:16: RuntimeWarning: overflow encountered in double_scalars
  return (-np.sum(p1 + p0))  + self.α * sum(np.square(θ[:]))
C:\ProgramData\Anaconda3\envs\tensorflow-22\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: overflow encountered in square
  return f_raw(*args, **kwargs)


In [ ]:
import numpy as np